## Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama (Persona)
## (Rule-Based Classification)

**İş Problemi**  

Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak seviye tabanlı (level based) yeni müşteri tanımları (persona) oluşturmak,
yeni müşteri tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.

Örneğin :
Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek kullanıcının ortalama ne kadar kazandırabileceği belirlenmek isteniyor.


**Veri Seti Hakkında**  

Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını ve bu ürünleri satın alan kullanıcıların bazı demografik bilgilerini barındırmaktadır. 

Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. 

Bunun anlamı tablo tekilleştirilmemiştir. 
Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla alışveriş yapmış olabilir.

**Değişkenler**

* PRICE : Müşterinin harcama tutarı
* SOURCE : Müşterinin bağlandığı cihaz türü
* SEX : Müşterinin cinsiyeti
* COUNTRY : Müşterinin ülkesi
* AGE : Müşterinin yaşı 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.max_rows",None)  # veri setinin okunaklı olması için

In [4]:
data = pd.read_csv("C:/Users/onurc/Documents/GitHub/dataScienceRepository/00_Data_Source/persona.csv")
df = data.copy()

In [7]:
#veri setinin ilk ve son 10 değerini inceliyoruz

df.head(10)
df.tail(10)

,PRICE,SOURCE,SEX,COUNTRY,AGE
4990,39,ios,male,usa,27
4991,29,ios,male,usa,27
4992,19,ios,male,usa,27
4993,39,ios,male,usa,27
4994,39,android,female,bra,31
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [8]:
df.shape

#veri setimiz 5000 satırdan ve 5 sütundan oluşmaktadır

(5000, 5)

In [9]:
df.info()

#veri setinde null değerimiz yok
## object olarak tutulan değerlere dikkat ediyoruz!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [10]:
#Kaç unique SOURCE vardır? Frekansları nedir?

df["SOURCE"].nunique()  ## 2 adet var => android ve ios olmak üzere
df["SOURCE"].value_counts() ## android ve ios cihazlarn. sayısı

android    2974
ios        2026
Name: SOURCE, dtype: int64

In [14]:
# Price değşk. segmentler görünüyor, bunları incelemekte fayda var

df["PRICE"].nunique()  # 6 adet fiyat segmenti görünüyor
df["PRICE"].value_counts() # hangi segmentten kaç adet olduğuna bakalım

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

In [15]:
#Hangi ülkeden kaç adet satış gerçekleştirilmiştir

df["COUNTRY"].value_counts()

#en çok satış USA (2065) ve BRA'da (1496) gerçekleşmiş
#bu ülkeleri en yakın takip eden 3. ülke ise 455 satış ile DEU

usa    2065
bra    1496
deu     455
tur     451
fra     303
can     230
Name: COUNTRY, dtype: int64

In [24]:
#Peki ülkere göre satışlardan toplam kazanç nedir?
#Persona oluştururken bunlara dikkat edeceğiz  (potansiyel hedef için)

df.groupby("COUNTRY").agg({"PRICE" : "sum"}).sort_values(by="PRICE", ascending=False)

#anomali yok 

,PRICE
COUNTRY,
usa,70225
bra,51354
tur,15689
deu,15485
fra,10177
can,7730


In [27]:
#Ülkelere göre PRICE ortalamaları nedir?

df.groupby("COUNTRY").agg({"PRICE" : "sum"}).sort_values(by="PRICE", ascending=False)

# toplam kazanç ve toplam satış adetleri incelendiğinde orantısı doğru 

,PRICE
COUNTRY,
usa,70225
bra,51354
tur,15689
deu,15485
fra,10177
can,7730


In [29]:
# SOURCE'lara göre PRICE ortalamaları nedir?

df.groupby("SOURCE").agg({"PRICE" : "mean"}).sort_values(by = "PRICE", ascending=False)

# fiyat ortalaması birbirine yakın görünüyor fakat 
# yüzdelik dilimde android kullanıcısı daha çoktur
# a.ortalama ve medyan arasındaki fark göz önünde bulundurulmalıdır. 

,PRICE
SOURCE,
android,34.174849
ios,34.069102


In [33]:
# COUNTRY-SOURCE kırılımında PRICE ortalamaları nedir?

df.groupby(["COUNTRY", "SOURCE"]).agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False)

,,PRICE
COUNTRY,SOURCE,
tur,android,36.229437
bra,android,34.387029
usa,ios,34.371703
fra,android,34.312500
deu,ios,34.268817
bra,ios,34.222222
can,ios,33.951456
deu,android,33.869888
usa,android,33.760357


In [38]:
#COUNTRY, SOURCE, SEX, AGE kırılımında ortalama kazançlar nedir

df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False).head()

# PRICE değerlerine göre azalan şekilde sıralandığından, 
# listenin üst sıralarındaki gruplar en yüksek ortalama kazançları ifade eder. 
# Bu gruplar işimiz için en değerli müşteri segmentlerini gösterebilir.

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [69]:
agg_df = df.groupby(["COUNTRY", "SOURCE", "SEX", "AGE"]).agg({"PRICE" : "mean"}).sort_values(by="PRICE", ascending=False)
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


In [70]:
# Yeni oluşturulan df index işlemleri

agg_df = agg_df.reset_index()
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


In [71]:
# AGE değişkenini kategorik değişkene çevireceğiz ve agg_df'e ekleyerek bütün oluşturacağız.
# bu işlemleri gerçekleştirirken personalar için aralıklar ve etiketler belirleyeceğiz.

agg_df["AGE"].describe()

myBins = [0, 18, 23, 30, 40, agg_df["AGE"].max()]    # bins = aralık
myLabels = ["0_18", "19_23", "24_30", "31_40", "41 -" + str(agg_df["AGE"].max())]    # labels = etiket, isimlendirme

agg_df["AGE_Category"] = pd.cut(agg_df["AGE"], bins = myBins, labels = myLabels)
agg_df.head()

# verileri gruplandırmak oldukça önemlidir, 
## bu yüzden yaş aralıklarına göre bir gruplandırma gerçekleştirdik
### bu sayede daha iyi bir gözlem gerçekleşir

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_Category
0,bra,android,male,46,59.0,41 -66
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


List comp customer_level_based değerleri oluşturulduktan sonra bu değerlerin tekilleştirilmesi gerekmektedir
Aksi durumda "ÇOKLAMA" gerçekleşir ve bu durum veriyi gözlemlemeyi zorlaştırır, veriyi manipüle eder.

Örn. birden fazla USA_ANDROID_MALE_0_18 değeri olabilir
Bu durumu düzeltmek için groupby kullanacağız    


In [72]:
# customer_level_based ile değerleri oluşturacağız ve bu değişkene atayacağız

agg_df["customers_level_based"] = agg_df[["COUNTRY", "SOURCE", "SEX", "AGE_Category"]].agg(lambda x: "_".join(x).upper(), axis=1)

agg_df = agg_df[["customers_level_based", "PRICE"]]

agg_df = agg_df.groupby("customers_level_based").agg({"PRICE": "mean"}).reset_index()

agg_df.head()


,customers_level_based,PRICE
0,BRA_ANDROID_FEMALE_0_18,35.645303
1,BRA_ANDROID_FEMALE_19_23,34.077340
2,BRA_ANDROID_FEMALE_24_30,33.863946
3,BRA_ANDROID_FEMALE_31_40,34.898326
4,BRA_ANDROID_FEMALE_41 -66,36.737179


In [75]:
# Yeni müşterileri (USA_ANDROID_MALE_0_18) PRICE'a göre segmentlere ayıralım

agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels=["D", "C", "B", "A"])

agg_df.groupby("SEGMENT").agg({"PRICE": "mean"}).sort_values(by="PRICE", ascending=False)

,PRICE
SEGMENT,
A,38.691234
B,34.999645
C,33.509674
D,29.206780


In [88]:
# Yeni gelen müşterileri sınıflandırarak ne kadar gelir getirebileceğini tahminleyelim.

# 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazandırması beklenir? 

new_user = "TUR_ANDROID_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]     # filtrelemek için


,customers_level_based,PRICE,SEGMENT
72,TUR_ANDROID_FEMALE_31_40,41.833333,A


In [87]:
# 35 yaşında IOS kullanan bir Fransız kadını hangi segmente ve ortalama ne kadar gelir kazandırması beklenir? 

new_user = "FRA_IOS_FEMALE_31_40"
agg_df[agg_df["customers_level_based"] == new_user]

,customers_level_based,PRICE,SEGMENT
63,FRA_IOS_FEMALE_31_40,32.818182,C
